In [7]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit.circuit import Parameter

# Fix font warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Liberation Sans', 'Arial', 'sans-serif']
plt.rcParams['mathtext.default'] = 'regular'

def create_xy_interaction(qc, qubit1, qubit2, beta_param):
    """Add a single XY interaction between two qubits."""
    qc.ry(np.pi/2, qubit1)      # Rotate qubit1 to X-Y plane
    qc.ry(np.pi/2, qubit2)      # Rotate qubit2 to X-Y plane
    qc.cx(qubit1, qubit2)       # First CNOT
    qc.rz(beta_param, qubit2)   # Z-rotation with beta parameter
    qc.cx(qubit1, qubit2)       # Second CNOT
    qc.ry(-np.pi/2, qubit1)     # Rotate qubit1 back
    qc.ry(-np.pi/2, qubit2)     # Rotate qubit2 back

def create_single_even_odd_layer(qc, beta_param):
    """Add a single complete even/odd layer to the circuit."""
    
    # EVEN LAYER: Connections starting from even indices
    qc.barrier(label='Even')
    
    # (0,1) - First even connection
    create_xy_interaction(qc, 0, 1, beta_param)
    
    # (2,3) - Second even connection  
    create_xy_interaction(qc, 2, 3, beta_param)
    
    # (4,5) - Third even connection
    create_xy_interaction(qc, 4, 5, beta_param)
    
    #qc.barrier()
    
    # ODD LAYER: Connections starting from odd indices
    qc.barrier(label='Odd')
    
    # (1,2) - First odd connection
    create_xy_interaction(qc, 1, 2, beta_param)
    
    # (3,4) - Second odd connection
    create_xy_interaction(qc, 3, 4, beta_param)
    
    # (5,0) - Third odd connection - CLOSES THE RING!
    create_xy_interaction(qc, 5, 0, beta_param)

def create_6qubit_even_odd_xy_mixer_trotterized(beta=np.pi/8, trotter_steps=2):
    """Create the 6-qubit even/odd XY mixer with Trotterization."""
    
    qc = QuantumCircuit(6, name=f'6Q_EvenOdd_XY_Mixer_Trotter{trotter_steps}')
    
    # Create symbolic parameter for display
    beta_param = Parameter('β')
    
    # Each Trotter step uses β/trotter_steps
    beta_per_step = 2 * beta_param / trotter_steps  # 2* because RZ gate needs 2*beta for XY
    
    # Apply trotter_steps repetitions of the even/odd pattern
    for step in range(trotter_steps):
        
        if step > 0:
            qc.barrier(label=f'Trotter {step+1}')
        
        # Apply one complete even/odd layer
        create_single_even_odd_layer(qc, beta_per_step)
        
        # Add barrier after each Trotter step (except the last)
        #if step < trotter_steps - 1:
        #    qc.barrier()
    
    # For simulation, we need to bind the parameter to actual values
    try:
        # Try assign_parameters method
        qc_bound = qc.assign_parameters({beta_param: beta})
    except AttributeError:
        # Fallback: create a separate numerical circuit
        qc_bound = QuantumCircuit(6, name=f'6Q_EvenOdd_XY_Mixer_Trotter{trotter_steps}_Bound')
        
        # Each Trotter step uses beta/trotter_steps
        beta_per_step_num = 2 * beta / trotter_steps
        
        for step in range(trotter_steps):
            if step > 0:
                qc_bound.barrier()
            
            # Even layer
            qc_bound.barrier()
            create_xy_interaction(qc_bound, 0, 1, beta_per_step_num)
            create_xy_interaction(qc_bound, 2, 3, beta_per_step_num)
            create_xy_interaction(qc_bound, 4, 5, beta_per_step_num)
            
            qc_bound.barrier()
            
            # Odd layer
            qc_bound.barrier()
            create_xy_interaction(qc_bound, 1, 2, beta_per_step_num)
            create_xy_interaction(qc_bound, 3, 4, beta_per_step_num)
            create_xy_interaction(qc_bound, 5, 0, beta_per_step_num)  # Closes the ring
            
            if step < trotter_steps - 1:
                qc_bound.barrier()
    
    return qc, qc_bound

def plot_6qubit_even_odd_xy_mixer_trotterized_single_line():
    """Plot the trotterized 6-qubit even/odd XY mixer in a single horizontal line."""
    
    qc_symbolic, qc_bound = create_6qubit_even_odd_xy_mixer_trotterized(trotter_steps=2)
    
    # Use symbolic circuit for display - force into single line (much longer!)
    try:
        # Method 1: Direct matplotlib circuit drawing - single line layout
        fig, ax = plt.subplots(figsize=(48, 8))  # Extra extra wide for trotterized version
        qc_symbolic.draw(output='mpl', ax=ax, style='default', 
                        fold=-1,  # Force no folding - single line
                        scale=0.4,  # Even smaller scale to fit trotterized circuit
                        plot_barriers=True)
        
        plt.tight_layout()
        plt.savefig('6qubit_even_odd_xy_mixer_trotterized_single_line.png', dpi=600, bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        
        print("✓ Saved circuit as '6qubit_even_odd_xy_mixer_trotterized_single_line.png' (Method 1)")
        plt.show()
        
    except Exception as e:
        print(f"Method 1 failed: {e}")
        
        # Method 2: Use circuit_drawer with single line settings
        try:
            plt.figure(figsize=(48, 8))  # Extra wide for trotterized landscape
            fig = circuit_drawer(qc_symbolic, output='mpl', style='default', 
                                fold=-1,  # Force no folding
                                scale=0.4,  # Smaller scale for trotterized
                                plot_barriers=True)
            
            if fig is not None:
                plt.savefig('6qubit_even_odd_xy_mixer_trotterized_single_line.png', dpi=600, bbox_inches='tight', 
                            facecolor='white', edgecolor='none')
                print("✓ Saved circuit as '6qubit_even_odd_xy_mixer_trotterized_single_line.png' (Method 2)")
                plt.show()
            else:
                print("Circuit drawer returned None")
                
        except Exception as e2:
            print(f"Method 2 also failed: {e2}")
            print("Falling back to text-only representation")
    
    # Always show text representation (truncated for readability)
    print("\nCircuit text representation (first 150 chars):")
    circuit_text = qc_symbolic.draw(output='text')
    print(circuit_text[:150] + "..." if len(circuit_text) > 150 else circuit_text)
    
    return qc_symbolic, qc_bound

def demonstrate_trotterized_6qubit_even_odd_effect():
    """Demonstrate what the trotterized 6-qubit even/odd XY mixer does."""
    
    try:
        # Try newer Qiskit first
        from qiskit_aer import Aer
        from qiskit import transpile
        use_aer = True
    except ImportError:
        try:
            # Try older Qiskit
            from qiskit import Aer
            from qiskit import transpile
            use_aer = True
        except ImportError:
            # Fall back to basic quantum_info
            from qiskit.quantum_info import Statevector
            use_aer = False
    
    print("\n6-Qubit Even/Odd XY Mixer - Trotterized Effect:")
    print("="*55)
    print("Using β = π/8 total (β/4 per Trotter step)")
    print("Testing ring connectivity with trotterized evolution")
    print()
    
    # Test state that should show ring mixing
    initial_state = '000001'  # Single excitation at position 0
    
    # Create circuit with initial state
    qc = QuantumCircuit(6)
    
    # Prepare initial state |000001⟩
    qc.x(0)  # Set qubit 0 to |1⟩
    
    # Add trotterized even/odd XY mixer
    _, xy_mixer_bound = create_6qubit_even_odd_xy_mixer_trotterized(beta=np.pi/8, trotter_steps=2)
    qc = qc.compose(xy_mixer_bound)
    
    if use_aer:
        # Use Aer backend
        backend = Aer.get_backend('statevector_simulator')
        transpiled_qc = transpile(qc, backend)
        job = backend.run(transpiled_qc)
        result = job.result()
        statevector = result.get_statevector()
    else:
        # Use quantum_info Statevector
        statevector = Statevector.from_instruction(qc)
    
    print(f"Initial state |{initial_state}⟩ after 2 Trotter steps:")
    
    # Display final state probabilities (top 8 most probable)
    probs = np.abs(statevector)**2
    top_indices = np.argsort(probs)[-8:][::-1]
    
    for idx in top_indices:
        if probs[idx] > 0.001:
            binary_state = format(idx, '06b')  # 6-bit binary
            print(f"  |{binary_state}⟩: {probs[idx]:.3f}")

def explain_trotterized_single_line_layout():
    """Explain the single line layout for the trotterized version."""
    
    print("\nTROTTERIZED SINGLE LINE LAYOUT:")
    print("="*40)
    
    print("Layout Specifications:")
    print("  • Figure size: 48×8 inches (6:1 aspect ratio!)")
    print("  • Scale: 0.4 (very compact for long circuit)")
    print("  • Fold: -1 (absolutely no line breaks)")
    print("  • Perfect for ultra-wide landscape slides")
    print()
    
    print("Trotterized Structure in Single Line:")
    print("  Step 1: Even → Odd | Step 2: Even → Odd")
    print("  All β parameters show as β (scaled by Trotter steps)")
    print()
    
    print("Visual Flow (Left to Right):")
    print("  [Even₁|Odd₁] → Barrier → [Even₂|Odd₂]")
    print("  Complete pattern repeated twice")
    print()
    
    print("Presentation Benefits:")
    print("  ✓ Shows complete trotterized evolution")
    print("  ✓ Clear repetition of even/odd pattern")
    print("  ✓ All parameters visible: β per step")
    print("  ✓ Perfect for explaining Trotterization concept")
    print("  ✓ Single view of entire mixing process")
    print()
    
    print("Ultra-Wide Slide Requirements:")
    print("  📐 Needs very wide slide format")
    print("  🔍 May need zoom for detailed viewing")
    print("  📊 Perfect for technical presentations")
    print("  🎯 Shows systematic approach clearly")

def main():
    """Generate and demonstrate the trotterized 6-qubit even/odd XY mixer in single line."""
    
    print("6-QUBIT EVEN/ODD XY MIXER - TROTTERIZED SINGLE LINE")
    print("="*65)
    print("Ultra-wide single line layout for landscape slides!")
    print("Creating trotterized 6-qubit even/odd XY mixer...")
    print("Layout: Complete trotterized circuit in one horizontal line")
    print("Structure: [Even+Odd] → [Even+Odd] with β/2 per step")
    print()
    
    # Plot the circuit in single line
    qc_symbolic, qc_bound = plot_6qubit_even_odd_xy_mixer_trotterized_single_line()
    
    # Demonstrate the effect
    demonstrate_trotterized_6qubit_even_odd_effect()
    
    # Explain single line benefits
    explain_trotterized_single_line_layout()
    
    print("\n" + "="*65)
    print("TROTTERIZED 6-QUBIT EVEN/ODD - PRESENTATION READY:")
    print("✓ Single ultra-wide line: 48×8 inch format")
    print("✓ Complete trotterized evolution visible")
    print("✓ Clear even/odd layer repetition")
    print("✓ All β parameters visible (scaled by steps)")
    print("✓ Perfect for explaining Trotterization concept")
    print("✓ Systematic approach clearly demonstrated")

if __name__ == "__main__":
    main()